In [1]:
import os
import io
import time
import pickle
import numpy as np
from numpy.random import choice, shuffle
import tqdm.notebook
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Nadam
from keras.layers import Dense, Activation, Dropout, Input, LeakyReLU
from keras.layers import Input, Embedding, Bidirectional, LSTM, GRU
from keras.models import Model, Sequential
from keras_self_attention import SeqSelfAttention

Using TensorFlow backend.


In [2]:
with open('pickle/all_song.pickle','rb') as read_file:
    all_song= pickle.load(read_file)
with open('pickle/notes_index_dict.pickle','rb') as read_file:
    notes_index_dict= pickle.load(read_file)

In [4]:
unique_notes = len(notes_index_dict)
seq_len = 50

In [7]:
class TrainModel:
  
    def __init__(self, epochs, train, target, batch_nnet_size, batch_song, optimizer, loss_fn,
                total_songs, model):
        self.epochs = epochs
        self.train = train
        self.target = target
        self.batch_nnet_size = batch_nnet_size
        self.batch_song = batch_song
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.total_songs = total_songs
        self.model = model
    
    def fit(self):
        for epoch in tqdm.notebook.tqdm(range(self.epochs),desc='epochs'):
            
            # for each epochs, we shuffle the list of all the datasets
            train_target = list(zip(self.train, self.target))
            shuffle(train_target)
            self.train, self.target = zip(*train_target)
            loss_total = 0
            steps = 0
            steps_nnet = 0

            # In each epoch, iterate all songs by the size of batch_song
            for i in tqdm.notebook.tqdm(range(0,self.total_songs, self.batch_song), desc='MUSIC'):

                steps += 1
                
                input_batch = [y for x in self.train[i:i+self.batch_song] for y in x]
                output_batch = [y for x in self.target[i:i+self.batch_song] for y in x]
                inputs_nnet_large = np.array(input_batch)
                outputs_nnet_large = np.array(output_batch)

                index_shuffled = np.arange(start=0, stop=len(inputs_nnet_large))
                np.random.shuffle(index_shuffled)
                
                # process the windows/target in batches
                for nnet_steps in tqdm.notebook.tqdm(range(0,len(index_shuffled),self.batch_nnet_size)):
                    steps_nnet += 1
                    current_index = index_shuffled[nnet_steps:nnet_steps+self.batch_nnet_size]

                    inputs_nnet, outputs_nnet = inputs_nnet_large[current_index], outputs_nnet_large[current_index]

                    # To make sure no exception thrown by tensorflow on autograph
                    if len(inputs_nnet) // self.batch_nnet_size != 1:
                        break
                    loss = self.train_step(inputs_nnet, outputs_nnet)
                    loss_total += tf.math.reduce_sum(loss)
                    if steps_nnet % 20 == 0:
                        print("epochs {} | Steps {} | total loss : {}".format(epoch + 1, steps_nnet,loss_total))


    @tf.function
    def train_step(self, inputs, targets):
        with tf.GradientTape() as tape:
            prediction = self.model(inputs)
            loss = self.loss_fn(targets, prediction)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return loss

In [8]:
train = []
target = []

for song in all_song:
    train.append(song['train'])
    target.append(song['target'])

In [16]:
train_windows = [y for x in train for y in x]
len(train_windows)

1493100

In [10]:
model = tf.keras.models.load_model('all_100sb_12ep.h5', custom_objects=SeqSelfAttention.get_custom_objects())

import the trained model with 12 epochs and keep training with 8 more epochs 

In [11]:
optimizer = Nadam()
loss_fn = sparse_categorical_crossentropy

In [12]:
seq_len = 50
EPOCHS = 8
BATCH_SONG = 100
BATCH_NNET_SIZE = 600
TOTAL_SONGS = len(all_song)

train_class = TrainModel(EPOCHS, train, target,
                  BATCH_NNET_SIZE, BATCH_SONG, optimizer, loss_fn,
                   TOTAL_SONGS, model)

train_class.fit()

epochs 1 | Steps 20 | total loss : 244.39230346679688
epochs 1 | Steps 40 | total loss : 519.43701171875
epochs 1 | Steps 60 | total loss : 770.0392456054688
epochs 1 | Steps 80 | total loss : 1006.257080078125
epochs 1 | Steps 100 | total loss : 1224.9437255859375
epochs 1 | Steps 120 | total loss : 1424.685791015625
epochs 1 | Steps 140 | total loss : 1646.9993896484375
epochs 1 | Steps 160 | total loss : 1848.6824951171875
epochs 1 | Steps 180 | total loss : 2054.56640625
epochs 1 | Steps 200 | total loss : 2414.7783203125
epochs 1 | Steps 220 | total loss : 2816.866455078125
epochs 1 | Steps 240 | total loss : 3339.47216796875
epochs 1 | Steps 260 | total loss : 3729.57275390625
epochs 1 | Steps 280 | total loss : 3982.76904296875
epochs 1 | Steps 300 | total loss : 4207.84423828125
epochs 1 | Steps 320 | total loss : 4393.15234375
epochs 1 | Steps 340 | total loss : 4577.1572265625
epochs 1 | Steps 360 | total loss : 4748.56689453125
epochs 1 | Steps 380 | total loss : 4952.643554

epochs 1 | Steps 800 | total loss : 8829.40625
epochs 1 | Steps 820 | total loss : 9037.9921875
epochs 1 | Steps 840 | total loss : 9216.8779296875
epochs 1 | Steps 860 | total loss : 9409.291015625
epochs 1 | Steps 880 | total loss : 9635.3779296875
epochs 1 | Steps 900 | total loss : 9850.220703125
epochs 1 | Steps 920 | total loss : 10022.15234375
epochs 1 | Steps 940 | total loss : 10248.6201171875
epochs 1 | Steps 960 | total loss : 10420.9716796875
epochs 1 | Steps 980 | total loss : 10596.404296875
epochs 1 | Steps 1000 | total loss : 10765.50390625
epochs 1 | Steps 1020 | total loss : 10924.798828125
epochs 1 | Steps 1040 | total loss : 11092.2626953125
epochs 1 | Steps 1060 | total loss : 11261.8359375
epochs 1 | Steps 1080 | total loss : 11418.54296875
epochs 1 | Steps 1100 | total loss : 11573.48046875
epochs 1 | Steps 1120 | total loss : 11730.5810546875
epochs 1 | Steps 1140 | total loss : 11894.7197265625
epochs 1 | Steps 1160 | total loss : 12050.521484375
epochs 1 | Ste

epochs 1 | Steps 1580 | total loss : 17487.46484375
epochs 1 | Steps 1600 | total loss : 17663.869140625
epochs 1 | Steps 1620 | total loss : 17837.849609375
epochs 1 | Steps 1640 | total loss : 18020.93359375
epochs 1 | Steps 1660 | total loss : 18185.349609375
epochs 1 | Steps 1680 | total loss : 18379.75
epochs 1 | Steps 1700 | total loss : 18553.09765625
epochs 1 | Steps 1720 | total loss : 18719.91015625
epochs 1 | Steps 1740 | total loss : 18896.482421875
epochs 1 | Steps 1760 | total loss : 19066.916015625
epochs 1 | Steps 1780 | total loss : 19230.001953125
epochs 1 | Steps 1800 | total loss : 19419.78125
epochs 1 | Steps 1820 | total loss : 19601.89453125
epochs 1 | Steps 1840 | total loss : 19779.962890625
epochs 1 | Steps 1860 | total loss : 19977.8125
epochs 1 | Steps 1880 | total loss : 20134.41796875
epochs 1 | Steps 1900 | total loss : 20316.75
epochs 1 | Steps 1920 | total loss : 20495.50390625
epochs 1 | Steps 1940 | total loss : 20669.7890625
epochs 1 | Steps 1960 | t

epochs 1 | Steps 2380 | total loss : 26074.453125
epochs 1 | Steps 2400 | total loss : 26273.203125
epochs 1 | Steps 2420 | total loss : 26471.365234375
epochs 1 | Steps 2440 | total loss : 26641.234375
epochs 1 | Steps 2460 | total loss : 26839.455078125
epochs 1 | Steps 2480 | total loss : 27039.5703125




epochs 2 | Steps 20 | total loss : 201.9156951904297
epochs 2 | Steps 40 | total loss : 364.16046142578125
epochs 2 | Steps 60 | total loss : 540.761474609375
epochs 2 | Steps 80 | total loss : 727.3439331054688
epochs 2 | Steps 100 | total loss : 888.2243041992188
epochs 2 | Steps 120 | total loss : 1060.4613037109375
epochs 2 | Steps 140 | total loss : 1235.9114990234375
epochs 2 | Steps 160 | total loss : 1392.9385986328125
epochs 2 | Steps 180 | total loss : 1559.8333740234375
epochs 2 | Steps 200 | total loss : 1746.76611328125
epochs 2 | Steps 220 | total loss : 1901.6368408203125
epochs 2 | Steps 240 | total loss : 2075.119384765625
epochs 2 | Steps 260 | total loss : 2249.10498046875
epochs 2 | Steps 280 | total loss : 2396.655029296875
epochs 2 | Steps 300 | total loss : 2537.251953125
epochs 2 | Steps 320 | total loss : 2719.81201171875
epochs 2 | Steps 340 | total loss : 2866.81298828125
epochs 2 | Steps 360 | total loss : 3056.556884765625
epochs 2 | Steps 380 | total loss 

...

epochs 7 | Steps 2380 | total loss : 22390.318359375
epochs 7 | Steps 2400 | total loss : 22612.017578125
epochs 7 | Steps 2420 | total loss : 22805.8125
epochs 7 | Steps 2440 | total loss : 22980.291015625
epochs 7 | Steps 2460 | total loss : 23177.900390625
epochs 7 | Steps 2480 | total loss : 23341.681640625




epochs 8 | Steps 20 | total loss : 169.2085418701172
epochs 8 | Steps 40 | total loss : 332.87451171875
epochs 8 | Steps 60 | total loss : 518.8392333984375
epochs 8 | Steps 80 | total loss : 699.2432861328125
epochs 8 | Steps 100 | total loss : 866.0827026367188
epochs 8 | Steps 120 | total loss : 1009.2328491210938
epochs 8 | Steps 140 | total loss : 1188.2490234375
epochs 8 | Steps 160 | total loss : 1340.93994140625
epochs 8 | Steps 180 | total loss : 1485.1510009765625
epochs 8 | Steps 200 | total loss : 1655.3572998046875
epochs 8 | Steps 220 | total loss : 1799.2252197265625
epochs 8 | Steps 240 | total loss : 1952.50830078125
epochs 8 | Steps 260 | total loss : 2129.179931640625
epochs 8 | Steps 280 | total loss : 2300.72509765625
epochs 8 | Steps 300 | total loss : 2476.623046875
epochs 8 | Steps 320 | total loss : 2616.0849609375
epochs 8 | Steps 340 | total loss : 2762.822509765625
epochs 8 | Steps 360 | total loss : 2909.07275390625
epochs 8 | Steps 380 | total loss : 3074.

epochs 8 | Steps 800 | total loss : 6458.89306640625
epochs 8 | Steps 820 | total loss : 6630.52099609375
epochs 8 | Steps 840 | total loss : 6803.5966796875
epochs 8 | Steps 880 | total loss : 7118.44384765625
epochs 8 | Steps 900 | total loss : 7278.328125
epochs 8 | Steps 920 | total loss : 7440.7568359375
epochs 8 | Steps 940 | total loss : 7586.8056640625
epochs 8 | Steps 960 | total loss : 7732.826171875
epochs 8 | Steps 980 | total loss : 7913.41552734375
epochs 8 | Steps 1000 | total loss : 8087.8505859375
epochs 8 | Steps 1020 | total loss : 8251.3974609375
epochs 8 | Steps 1040 | total loss : 8376.6806640625
epochs 8 | Steps 1060 | total loss : 8544.8837890625
epochs 8 | Steps 1080 | total loss : 8709.3779296875
epochs 8 | Steps 1100 | total loss : 8856.6591796875
epochs 8 | Steps 1120 | total loss : 9020.619140625
epochs 8 | Steps 1140 | total loss : 9163.3935546875
epochs 8 | Steps 1160 | total loss : 9306.037109375
epochs 8 | Steps 1180 | total loss : 9464.259765625
epochs

epochs 8 | Steps 1580 | total loss : 13235.3134765625
epochs 8 | Steps 1600 | total loss : 13401.2548828125
epochs 8 | Steps 1620 | total loss : 13541.826171875
epochs 8 | Steps 1640 | total loss : 13728.6962890625
epochs 8 | Steps 1680 | total loss : 14071.3134765625
epochs 8 | Steps 1700 | total loss : 14237.7080078125
epochs 8 | Steps 1720 | total loss : 14409.5087890625
epochs 8 | Steps 1740 | total loss : 14570.625
epochs 8 | Steps 1760 | total loss : 14711.2978515625
epochs 8 | Steps 1780 | total loss : 14863.49609375
epochs 8 | Steps 1800 | total loss : 15024.45703125
epochs 8 | Steps 1820 | total loss : 15179.24609375
epochs 8 | Steps 1840 | total loss : 15347.4873046875
epochs 8 | Steps 1860 | total loss : 15566.5107421875
epochs 8 | Steps 1880 | total loss : 15743.3603515625
epochs 8 | Steps 1900 | total loss : 15927.2275390625
epochs 8 | Steps 1920 | total loss : 16072.865234375
epochs 8 | Steps 1940 | total loss : 16222.6513671875
epochs 8 | Steps 1960 | total loss : 16377.

epochs 8 | Steps 2380 | total loss : 19910.82421875
epochs 8 | Steps 2400 | total loss : 20082.05078125
epochs 8 | Steps 2420 | total loss : 20252.294921875
epochs 8 | Steps 2440 | total loss : 20417.7109375
epochs 8 | Steps 2460 | total loss : 20573.12890625
epochs 8 | Steps 2480 | total loss : 20763.814453125





In [14]:
model.save('all_100sb_20ep.h5')